In [2]:
import numpy as np
import pandas as pd

In [55]:
#Read data & print the head
df = pd.read_csv('fordBike_tripData_201901.csv')
df.head()

,duration_sec,start_time,start_station_id,start_station_name,end_station_id,end_station_name,user_type,bike_share_for_all_trip
0,226,2019-01-01 00:07:10.576 UTC,245.0,Downtown Berkeley BART,247.0,Fulton St at Bancroft Way,Customer,No
1,1767,2019-01-01 00:08:39.659 UTC,16.0,Steuart St at Market St,115.0,Jackson Playground,Customer,No
2,182,2019-01-01 00:09:56.211 UTC,269.0,Telegraph Ave at Carleton St,240.0,Haste St at Telegraph Ave,Subscriber,Yes
3,565,2019-01-01 00:11:03.441 UTC,269.0,Telegraph Ave at Carleton St,243.0,Bancroft Way at College Ave,Subscriber,Yes
4,710,2019-01-01 00:14:48.398 UTC,163.0,Lake Merritt BART Station,159.0,24th St at Market St,Customer,No


In [41]:
# Get the one-way route volume for all trips

# Concatenate start and end stations to count how many trips were made with that combination
df["start_station|end_station"] = df["start_station_id"].map(str) +"|" + df["end_station_id"].map(str)    
df["route"] = df["start_station_name"].map(str) +"|" + df["end_station_name"].map(str)
df["route_volume"] = df["start_station|end_station"]

# Get the volume count for each route
route_volume_series = df["start_station|end_station"].value_counts()
route_volume_dict = route_volume_series.to_dict()

# Write the volume number to each row in the DF
df['route_volume'] = df['route_volume'].map(route_volume_dict) 

# Sort DF by route volume
df_volume_sort = df[['start_station|end_station', 'route', 'route_volume']]
df_volume_sort = df_volume_sort.sort_values(by='route_volume', ascending=False)
df_volume_sort.head()


,start_station|end_station,route,route_volume
109977,15.0|6.0,San Francisco Ferry Building (Harry Bridges Pl...,453
124182,15.0|6.0,San Francisco Ferry Building (Harry Bridges Pl...,453
68163,15.0|6.0,San Francisco Ferry Building (Harry Bridges Pl...,453
50167,15.0|6.0,San Francisco Ferry Building (Harry Bridges Pl...,453
67124,15.0|6.0,San Francisco Ferry Building (Harry Bridges Pl...,453


In [50]:
# To group common routes in the same cluster, make a column for the reciprical of route volume. 
# We will cluster on the reciprical to promote high volume routes being clustered together.
df_volume_sort['route_volume_reciprical'] = 1/df_volume_sort['route_volume']

df_volume_cluster = df_volume_sort[['route_volume_reciprical']]
df_volume_cluster.head()

,route_volume_reciprical
109977,0.002208
124182,0.002208
68163,0.002208
50167,0.002208
67124,0.002208


## Testing the clustering stuff below...in progress.

In [56]:
# Cluster by route_volume_reciprical

from sklearn.cluster import AgglomerativeClustering
import warnings
warnings.filterwarnings('ignore')

# Create clustering object
cluster_model = AgglomerativeClustering(df_volume_cluster)
# df_volume_cluster['Cluster'] = cluster_model

# test = np.array([[1, 0.5], [2, .75]])

# print(cluster_model.fit_predict(test))

df_volume_cluster.head()


,route_volume_reciprical
109977,0.002208
124182,0.002208
68163,0.002208
50167,0.002208
67124,0.002208
